In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import librosa
from IPython.display import Audio
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
import tensorflow as tf

In [2]:
train_folder_path = r'C:../dataset/audioonly/labeled/set 1/train'
test_folder_path = r'C:../dataset/audioonly/labeled/set 1/test'

In [3]:
os.path.exists(train_folder_path)

True

In [4]:
class_names = next(os.walk(train_folder_path))[1]
class_names

['belly_pain', 'discomfort', 'hungry', 'tired']

In [5]:
audio_path = [os.path.join(train_folder_path, name) for name in class_names]
audio_path

['C:../dataset/audioonly/labeled/set 1/train\\belly_pain',
 'C:../dataset/audioonly/labeled/set 1/train\\discomfort',
 'C:../dataset/audioonly/labeled/set 1/train\\hungry',
 'C:../dataset/audioonly/labeled/set 1/train\\tired']

In [6]:
bpain_audio = glob.glob(os.path.join(audio_path[0], '*.wav'))
discomf_audio = glob.glob(os.path.join(audio_path[1], '*.wav'))
hungry_audio = glob.glob(os.path.join(audio_path[2], '*.wav'))
tired_audio = glob.glob(os.path.join(audio_path[3], '*.wav'))

In [7]:
audio_path_byclass = {
    'bpain': bpain_audio,
    'discomf': discomf_audio,
    'hungry': hungry_audio,
    'tired': tired_audio
}

## Load Audio

In [8]:
# Replace 'your_audio_file.mp3' with the path to your audio file
load_audio = {}
for name, pathlst in audio_path_byclass.items():
    loaded = []
    for path in pathlst:
        audio, sr = librosa.load(path, sr=16000, mono=True, duration=5.2)
        loaded.append(audio)
    load_audio[name] = loaded

## Load Model

In [9]:
# Load the model.
import tensorflow_hub as hub
model = hub.load('https://tfhub.dev/google/yamnet/1')

In [10]:
# Run the model, check the output.
feature = {}
for name, arr in load_audio.items():
    # Initialize dictionaries for each class if not already existing
    if name not in feature:
        feature[name] = {'scores': [], 'embeddings': [], 'spectrograms': []}
    
    for wav in arr:
        scores, embeddings, spectrogram = model(wav)
        
        # Append the results to the respective lists in the dictionary
        feature[name]['scores'].append(np.array(scores))
        feature[name]['embeddings'].append(np.array(embeddings))
        feature[name]['spectrograms'].append(np.array(spectrogram))

In [11]:
import collections

shape = {}

for class_name in feature.keys():
    for cat, lst in feature[class_name].items():
        shapes = [arr.shape for arr in lst]
        shapes_count = collections.Counter(shapes)
        shape[f'{class_name} {cat} shape counts'] = shapes_count

In [12]:
shape

{'bpain scores shape counts': Counter({(10, 521): 16}),
 'bpain embeddings shape counts': Counter({(10, 1024): 16}),
 'bpain spectrograms shape counts': Counter({(528, 64): 16}),
 'discomf scores shape counts': Counter({(10, 521): 64}),
 'discomf embeddings shape counts': Counter({(10, 1024): 64}),
 'discomf spectrograms shape counts': Counter({(528, 64): 64}),
 'hungry scores shape counts': Counter({(10, 521): 100}),
 'hungry embeddings shape counts': Counter({(10, 1024): 100}),
 'hungry spectrograms shape counts': Counter({(528, 64): 100}),
 'tired scores shape counts': Counter({(10, 521): 24}),
 'tired embeddings shape counts': Counter({(10, 1024): 24}),
 'tired spectrograms shape counts': Counter({(528, 64): 24})}

## nd array padding
- 각 클래스 카테고리마다 평균 row를 구한 후 padding 해준다

In [13]:
from statistics import mean

temp = []

for class_cat, counter in shape.items():
    for shape, count in counter.items():
        h, w = shape
        num_elements = h * w
        temp.append(num_elements)

In [14]:
avg = mean(temp)

In [15]:
avg

16414

In [16]:
reshaped = {}

for class_name, categories in feature.items():
    for cat, lst in categories.items():
        temp = []
        for arr in lst:
            h, w = arr.shape
            flat_arr = arr.flatten()
            size = h * w

            if size == 5210 or size == 10240:
                padded = np.pad(flat_arr, pad_width=(0, avg - size), mode='constant', constant_values=0)
                temp.append(padded)
            elif size == 33792:
                trunc_arr = flat_arr[:avg]
                temp.append(trunc_arr)
        
        reshaped[f'{class_name} {cat}'] = temp

In [17]:
shape_count = {}

for class_name, lst in reshaped.items():
    shapes = [arr.shape for arr in lst]
    shapes_count = collections.Counter(shapes)
    shape_count[f'{class_name} shape counts'] = shapes_count

In [18]:
shape_count

{'bpain scores shape counts': Counter({(16414,): 16}),
 'bpain embeddings shape counts': Counter({(16414,): 16}),
 'bpain spectrograms shape counts': Counter({(16414,): 16}),
 'discomf scores shape counts': Counter({(16414,): 64}),
 'discomf embeddings shape counts': Counter({(16414,): 64}),
 'discomf spectrograms shape counts': Counter({(16414,): 64}),
 'hungry scores shape counts': Counter({(16414,): 100}),
 'hungry embeddings shape counts': Counter({(16414,): 100}),
 'hungry spectrograms shape counts': Counter({(16414,): 100}),
 'tired scores shape counts': Counter({(16414,): 24}),
 'tired embeddings shape counts': Counter({(16414,): 24}),
 'tired spectrograms shape counts': Counter({(16414,): 24})}

In [19]:
reshaped.keys()

dict_keys(['bpain scores', 'bpain embeddings', 'bpain spectrograms', 'discomf scores', 'discomf embeddings', 'discomf spectrograms', 'hungry scores', 'hungry embeddings', 'hungry spectrograms', 'tired scores', 'tired embeddings', 'tired spectrograms'])

In [20]:
# Define your class names
class_names = ['bpain', 'discomf', 'hungry', 'tired']

# Initialize dictionaries for each category
score_only = {class_name: [] for class_name in class_names}
embed_only = {class_name: [] for class_name in class_names}
spec_only = {class_name: [] for class_name in class_names}

# Loop over reshaped items only once
for category, lists in reshaped.items():
    for class_name in class_names:
        if f'{class_name} scores' in category:
            score_only[class_name].extend(lists)
        elif f'{class_name} embeddings' in category:
            embed_only[class_name].extend(lists)
        elif f'{class_name} spectrograms' in category:
            spec_only[class_name].extend(lists)

In [21]:
bpain_score_arr = np.array(score_only['bpain'])
discomf_score_arr = np.array(score_only['discomf'])
hungry_score_arr = np.array(score_only['hungry'])
tired_score_arr = np.array(score_only['tired'])
tired_score_arr.shape

(24, 16414)

In [22]:
score_stack = np.vstack((bpain_score_arr, discomf_score_arr, hungry_score_arr, tired_score_arr))
score_stack.shape

(204, 16414)

In [23]:
if np.array_equal(score_stack[180:204, :], tired_score_arr):
    print('matching true')
else:
    print('matching false')

matching true


In [24]:
bpain_embed_arr = np.array(embed_only['bpain'])
discomf_embed_arr = np.array(embed_only['discomf'])
hungry_embed_arr = np.array(embed_only['hungry'])
tired_embed_arr = np.array(embed_only['tired'])

In [25]:
embed_stack = np.vstack((bpain_embed_arr, discomf_embed_arr, hungry_embed_arr, tired_embed_arr))
embed_stack.shape

(204, 16414)

In [26]:
bpain_spec_arr = np.array(spec_only['bpain'])
discomf_spec_arr = np.array(spec_only['discomf'])
hungry_spec_arr = np.array(spec_only['hungry'])
tired_spec_arr = np.array(spec_only['tired'])

In [27]:
spec_stack = np.vstack((bpain_spec_arr, discomf_spec_arr, hungry_spec_arr, tired_spec_arr))
spec_stack.shape

(204, 16414)

In [28]:
# Combine data and labels
score_labels = []

score_arrlst = [
    bpain_score_arr,
    discomf_score_arr,
    hungry_score_arr,
    tired_score_arr
]

for idx, arr in enumerate(score_arrlst):
    score_labels.extend([idx] * len(arr))

y_train = np.array(score_labels)
y_train.shape

(204,)

In [29]:
for i in range(4):
    print(score_labels.count(i))

16
64
100
24


In [30]:
# Generate shuffled indices
shuffled_indices = np.random.permutation(score_stack.shape[0])

# Shuffle each array using the same shuffled indices
shuffled_score = score_stack[shuffled_indices, :]
shuffled_embed = embed_stack[shuffled_indices, :]
shuffled_spec = spec_stack[shuffled_indices, :]
y_train = y_train[shuffled_indices]

## Functional API Tried

In [4]:
from tensorflow import keras
from keras import layers
from keras import models, utils